In [3]:
import random

In [10]:
random.uniform(-1,1)

0.15922758347253296

In [1]:
import random
import mujoco as mj

max_width = 100
max_height = 100
#ctx = mj.GLContext(max_width, max_height)
#ctx.make_current()

cam = mj.MjvCamera()
opt = mj.MjvOption()

mj.glfw.glfw.init()
window = mj.glfw.glfw.create_window(1200, 900, "Demo", None, None)
mj.glfw.glfw.make_context_current(window)
mj.glfw.glfw.swap_interval(1)

mj.mjv_defaultCamera(cam)
mj.mjv_defaultOption(opt)

#cam.distance = 1

xml_path = "./model/composite/particle-big.xml"
#xml_path = "/home/jack/dm_control/dm_control/suite/quadruped.xml"
model = mj.MjModel.from_xml_path(xml_path)
data = mj.MjData(model)


def control(data):
    #for i in range(0, len(data.qvel), 6):
        #data.qvel[0 + i] = 0.001
        #data.qvel[1 + i] = 0.001
    
    data.ctrl[0] = random.uniform(0, 0.001)
    data.ctrl[1] = 0
    data.ctrl[2] = -0.001
    data.ctrl[3] = -0.001
    pass
        
        
scene = mj.MjvScene(model, maxgeom=10000)
context = mj.MjrContext(model, mj.mjtFontScale.mjFONTSCALE_150.value)
#count_out=0
while not mj.glfw.glfw.window_should_close(window):
    simstart = data.time
    count_in = 0
    while (data.time - simstart < 1.0/60.0):
        # We update our environment here
        mj.mj_step(model, data)
        
        #data.qvel[1] += 0.001 * random.uniform(-1,1)
        #data.qvel[0] += 0.001 * random.uniform(-1,1)
        
        #data.qvel[1] += 0.001 * random.uniform(-1,1)
        #print("Time:", data.time)
        
        control(data)
        cam.distance += 0.00001
        #count_in += 1
    
    #print(count_in, count_out)
    
    #print(data.time)
    #viewport = mj.MjrRect(0, 0, 0, 0)
    #mj.glfw.glfw.get_framebuffer_size(window)
    viewport = mj.MjrRect(0, 0, 1200, 900)

    #mj.mjv_updateScene(model, data, opt, None, cam, 0, scene)
    mj.mjv_updateScene(model, data, opt, None, cam, mj.mjtCatBit.mjCAT_ALL.value, scene)
    mj.mjr_render(viewport, scene, context)

    mj.glfw.glfw.swap_buffers(window)
    mj.glfw.glfw.poll_events()
    
    #count_out += 1

mj.glfw.glfw.terminate()

In [ ]:
data.actuator_force

In [3]:
data.ctrl[0] = 1

IndexError: index 0 is out of bounds for axis 0 with size 0

In [1]:
from dm_control import mjcf

class Arm(object):

  def __init__(self, name):
    self.mjcf_model = mjcf.RootElement(model=name)

    self.upper_arm = self.mjcf_model.worldbody.add('body', name='upper_arm')
    self.shoulder = self.upper_arm.add('joint', name='shoulder', type='ball')
    self.upper_arm.add('geom', name='upper_arm', type='capsule',
                       pos=[0, 0, -0.15], size=[0.045, 0.15])

    self.forearm = self.upper_arm.add('body', name='forearm', pos=[0, 0, -0.3])
    self.elbow = self.forearm.add('joint', name='elbow',
                                  type='hinge', axis=[0, 1, 0])
    self.forearm.add('geom', name='forearm', type='capsule',
                     pos=[0, 0, -0.15], size=[0.045, 0.15])

class UpperBody(object):

  def __init__(self):
    self.mjcf_model = mjcf.RootElement()
    self.mjcf_model.worldbody.add(
        'geom', name='torso', type='box', size=[0.15, 0.045, 0.25])
    left_shoulder_site = self.mjcf_model.worldbody.add(
        'site', size=[1e-6]*3, pos=[-0.15, 0, 0.25])
    right_shoulder_site = self.mjcf_model.worldbody.add(
        'site', size=[1e-6]*3, pos=[0.15, 0, 0.25])

    self.left_arm = Arm(name='left_arm')
    left_shoulder_site.attach(self.left_arm.mjcf_model)

    self.right_arm = Arm(name='right_arm')
    right_shoulder_site.attach(self.right_arm.mjcf_model)

body = UpperBody()
physics = mjcf.Physics.from_mjcf_model(body.mjcf_model)

In [2]:
suite.BENCHMARKING

(('acrobot', 'swingup'),
 ('acrobot', 'swingup_sparse'),
 ('ball_in_cup', 'catch'),
 ('cartpole', 'balance'),
 ('cartpole', 'balance_sparse'),
 ('cartpole', 'swingup'),
 ('cartpole', 'swingup_sparse'),
 ('cheetah', 'run'),
 ('finger', 'spin'),
 ('finger', 'turn_easy'),
 ('finger', 'turn_hard'),
 ('fish', 'upright'),
 ('fish', 'swim'),
 ('hopper', 'stand'),
 ('hopper', 'hop'),
 ('humanoid', 'stand'),
 ('humanoid', 'walk'),
 ('humanoid', 'run'),
 ('manipulator', 'bring_ball'),
 ('pendulum', 'swingup'),
 ('point_mass', 'easy'),
 ('reacher', 'easy'),
 ('reacher', 'hard'),
 ('swimmer', 'swimmer6'),
 ('swimmer', 'swimmer15'),
 ('walker', 'stand'),
 ('walker', 'walk'),
 ('walker', 'run'))

In [40]:
%load_ext autoreload
%autoreload 2

from dm_control_local import suite
import numpy as np
from PIL import Image
import subprocess



def get_action(total_observation, timestep, action_spec, chrom):
    if total_observation == None:
        action = np.random.uniform(action_spec.minimum,
                               action_spec.maximum,
                               size=action_spec.shape)
    else:
        act_dim = action_spec.shape[0]
        
        # 2d vector to target
        x_in = total_observation.observation['to_target']
        
        # Sin wave of N timesteps per cicle (agent would learn wavy-patterns)
        cycle_len = 10 # in timesteps
        t_in = np.sin(timestep / cycle_len * 2 * np.pi)
        
        # Concatenate Bias + Input values
        x_in = np.concatenate([[1, t_in], x_in])
        
        x_in_dim = len(x_in)
        # Todo: completar con la combinacion lineal de x_in contra parametros dados en chrom? o con algo intermedio
        #       notar que todavia no puedo llamar la funcion con chrom porque no existe chrom en este archivo.

        M = np.ones((x_in_dim, act_dim))*0.01
        
        #print("x_in:", x_in)
        
        action = np.tanh( np.matmul(x_in, M) )
        
        
    return action

# Load one task:
env = suite.load('swimmer', 'swimmer6', visualize_reward=True)



# Iterate over a task set:
# for domain_name, task_name in [suite.BENCHMARKING[0]]:
#     env = suite.load(domain_name, task_name)

# Step through an episode and print out reward, discount and observation.

total_observation = env.reset()
action_spec = env.action_spec()


timestep = 0
i = 0
idx = 0
action = np.asarray([0.] * 5)

while not total_observation.last():
    chrom = None # TODO Get chromosome array or matrix
    action = get_action(total_observation, timestep, action_spec, chrom)
    
    print("action: ", action)
    
    idx = (idx + 1) % 5
    total_observation = env.step(action)
    pixels = env.physics.render()#camera_id="back"
    im = Image.fromarray(pixels, 'RGB')
    # im.show()
    if timestep % 10 == 0:
        im.save("frames/frame-%.8d.png" % i)
        i += 1
    timestep += 1
    
    #print(total_observation)

    
    # 0: perfect score (0 distance to target)
    #score = - np.linalg.norm(total_observation.observation['to_target'])
    reward = total_observation.reward
    # creating a object
    #im = Image.open(r"C:\Users\System-Pc\Desktop\home.png")
    #print(i, total_observation.reward)#, total_observation.discount, total_observation.observation)
    #print(count, score, "\n", total_observation.observation)#, total_observation.discount, total_observation.observation)
    


subprocess.call([
    'ffmpeg', '-framerate', '5', '-y', '-i', 'frames/frame-%08d.png', '-r', '30', '-pix_fmt', 'yuv420p', 'video_name.mp4'
])

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
x_in: [ 1.          0.         -1.18725037 -0.44896057]
action:  [-0.00636202 -0.00636202 -0.00636202 -0.00636202 -0.00636202]
x_in: [ 1.          0.58778525 -1.1872896  -0.44887225]
action:  [-0.00048377 -0.00048377 -0.00048377 -0.00048377 -0.00048377]
x_in: [ 1.          0.95105652 -1.18736607 -0.44870001]
action:  [0.00314989 0.00314989 0.00314989 0.00314989 0.00314989]
x_in: [ 1.          0.95105652 -1.18742565 -0.44856578]
action:  [0.00315064 0.00315064 0.00315064 0.00315064 0.00315064]
x_in: [ 1.          0.58778525 -1.18744879 -0.44851363]
action:  [-0.00048177 -0.00048177 -0.00048177 -0.00048177 -0.00048177]
x_in: [ 1.00000000e+00  1.22464680e-16 -1.18745790e+00 -4.48493097e-01]
action:  [-0.00635942 -0.00635942 -0.00635942 -0.00635942 -0.00635942]
x_in: [ 1.         -0.58778525 -1.18750879 -0.44837839]
action:  [-0.01223611 -0.01223611 -0.01223611 -0.01223611 -0.01223611]
x_in: [ 1.       

x_in: [ 1.         -0.58778525 -1.27610662 -0.15991701]
action:  [-0.01023773 -0.01023773 -0.01023773 -0.01023773 -0.01023773]
x_in: [ 1.         -0.95105652 -1.27703144 -0.15475936]
action:  [-0.01382759 -0.01382759 -0.01382759 -0.01382759 -0.01382759]
x_in: [ 1.         -0.95105652 -1.27798036 -0.14934194]
action:  [-0.01378292 -0.01378292 -0.01378292 -0.01378292 -0.01378292]
x_in: [ 1.         -0.58778525 -1.27895667 -0.14362571]
action:  [-0.01010333 -0.01010333 -0.01010333 -0.01010333 -0.01010333]
x_in: [ 1.00000000e+00 -1.95943488e-15 -1.27994334e+00 -1.37688331e-01]
action:  [-0.00417629 -0.00417629 -0.00417629 -0.00417629 -0.00417629]
x_in: [ 1.          0.58778525 -1.28091052 -0.13169512]
action:  [0.00175179 0.00175179 0.00175179 0.00175179 0.00175179]
x_in: [ 1.          0.95105652 -1.28182771 -0.12583609]
action:  [0.00543387 0.00543387 0.00543387 0.00543387 0.00543387]
x_in: [ 1.          0.95105652 -1.28267516 -0.1202538 ]
action:  [0.00548122 0.00548122 0.00548122 0.0054

x_in: [ 1.         -0.58778525 -1.29201432  0.18405109]
action:  [-0.00695737 -0.00695737 -0.00695737 -0.00695737 -0.00695737]
x_in: [ 1.00000000e+00 -1.07793678e-14 -1.29166438e+00  1.88028232e-01]
action:  [-0.00103636 -0.00103636 -0.00103636 -0.00103636 -0.00103636]
x_in: [ 1.          0.58778525 -1.29129792  0.19204994]
action:  [0.00488533 0.00488533 0.00488533 0.00488533 0.00488533]
x_in: [ 1.          0.95105652 -1.29093437  0.19590762]
action:  [0.00856009 0.00856009 0.00856009 0.00856009 0.00856009]
x_in: [ 1.          0.95105652 -1.29059069  0.19944353]
action:  [0.00859888 0.00859888 0.00859888 0.00859888 0.00859888]
x_in: [ 1.          0.58778525 -1.29027434  0.20260997]
action:  [0.00500117 0.00500117 0.00500117 0.00500117 0.00500117]
x_in: [ 1.00000000e+00  7.34911876e-15 -1.28997985e+00  2.05486986e-01]
action:  [-0.00084493 -0.00084493 -0.00084493 -0.00084493 -0.00084493]
x_in: [ 1.         -0.58778525 -1.28969057  0.20825234]
action:  [-0.00669213 -0.00669213 -0.006692

x_in: [ 1.          0.58778525 -1.26667951  0.34974282]
action:  [0.00670839 0.00670839 0.00670839 0.00670839 0.00670839]
x_in: [ 1.          0.95105652 -1.26636618  0.35115622]
action:  [0.0103581 0.0103581 0.0103581 0.0103581 0.0103581]
x_in: [ 1.          0.95105652 -1.26611903  0.35226281]
action:  [0.01037163 0.01037163 0.01037163 0.01037163 0.01037163]
x_in: [ 1.          0.58778525 -1.26595085  0.35300871]
action:  [0.00674833 0.00674833 0.00674833 0.00674833 0.00674833]
x_in: [ 1.00000000e+00  1.95819692e-15 -1.26584693e+00  3.53463199e-01]
action:  [0.00087616 0.00087616 0.00087616 0.00087616 0.00087616]
x_in: [ 1.         -0.58778525 -1.2657706   0.35379282]
action:  [-0.00499759 -0.00499759 -0.00499759 -0.00499759 -0.00499759]
x_in: [ 1.         -0.95105652 -1.26567719  0.35419873]
action:  [-0.00862514 -0.00862514 -0.00862514 -0.00862514 -0.00862514]
x_in: [ 1.         -0.95105652 -1.26553114  0.35484004]
action:  [-0.00861726 -0.00861726 -0.00861726 -0.00861726 -0.00861726

x_in: [ 1.         -0.58778525 -1.27095204  0.3282207 ]
action:  [-0.00530512 -0.00530512 -0.00530512 -0.00530512 -0.00530512]
x_in: [ 1.00000000e+00 -2.15587355e-14 -1.27110952e+00  3.27421123e-01]
action:  [0.00056312 0.00056312 0.00056312 0.00056312 0.00056312]
x_in: [ 1.          0.58778525 -1.27125085  0.32669971]
action:  [0.00643225 0.00643225 0.00643225 0.00643225 0.00643225]
x_in: [ 1.          0.95105652 -1.27141657  0.32585377]
action:  [0.0100546 0.0100546 0.0100546 0.0100546 0.0100546]
x_in: [ 1.          0.95105652 -1.2716381   0.32472373]
action:  [0.01004108 0.01004108 0.01004108 0.01004108 0.01004108]
x_in: [ 1.          0.58778525 -1.27192539  0.32325518]
action:  [0.00639106 0.00639106 0.00639106 0.00639106 0.00639106]
x_in: [ 1.00000000e+00  3.91763180e-15 -1.27226306e+00  3.21520433e-01]
action:  [0.00049257 0.00049257 0.00049257 0.00049257 0.00049257]
x_in: [ 1.         -0.58778525 -1.27261672  0.31969076]
action:  [-0.00540706 -0.00540706 -0.00540706 -0.00540706 

...

x_in: [ 1.         -0.58778525 -1.28220807  0.25469533]
action:  [-0.0061529 -0.0061529 -0.0061529 -0.0061529 -0.0061529]
x_in: [ 1.00000000e+00 -3.91961253e-14 -1.28213335e+00  2.55198880e-01]
action:  [-0.00026934 -0.00026934 -0.00026934 -0.00026934 -0.00026934]
x_in: [ 1.          0.58778525 -1.2820436   0.25580625]
action:  [0.00561542 0.00561542 0.00561542 0.00561542 0.00561542]
x_in: [ 1.          0.95105652 -1.28196829  0.25631079]
action:  [0.00925373 0.00925373 0.00925373 0.00925373 0.00925373]
x_in: [ 1.          0.95105652 -1.28193099  0.25654904]
action:  [0.00925648 0.00925648 0.00925648 0.00925648 0.00925648]
x_in: [ 1.          0.58778525 -1.28194027  0.25646229]
action:  [0.00562301 0.00562301 0.00562301 0.00562301 0.00562301]
x_in: [ 1.00000000e+00 -6.86668780e-15 -1.28198627e+00  2.56119015e-01]
action:  [-0.00025867 -0.00025867 -0.00025867 -0.00025867 -0.00025867]
x_in: [ 1.         -0.58778525 -1.2820445   0.25568939]
action:  [-0.00614133 -0.00614133 -0.00614133 -0

x_in: [ 1.         -0.95105652 -1.27910238  0.27400396]
action:  [-0.00956126 -0.00956126 -0.00956126 -0.00956126 -0.00956126]
x_in: [ 1.         -0.58778525 -1.27900804  0.2745828 ]
action:  [-0.00592204 -0.00592204 -0.00592204 -0.00592204 -0.00592204]
x_in: [ 1.00000000e+00 -1.95943488e-14 -1.27887189e+00  2.75423916e-01]
action:  [-3.44796944e-05 -3.44796944e-05 -3.44796944e-05 -3.44796944e-05
 -3.44796944e-05]
x_in: [ 1.          0.58778525 -1.27872053  0.27635693]
action:  [0.00585415 0.00585415 0.00585415 0.00585415 0.00585415]
x_in: [ 1.          0.95105652 -1.27858657  0.27717682]
action:  [0.00949618 0.00949618 0.00949618 0.00949618 0.00949618]
x_in: [ 1.          0.95105652 -1.27849613  0.27772216]
action:  [0.00950254 0.00950254 0.00950254 0.00950254 0.00950254]
x_in: [ 1.          0.58778525 -1.27845874  0.27793575]
action:  [0.00587256 0.00587256 0.00587256 0.00587256 0.00587256]
x_in: [ 1.00000000e+00  3.03749545e-14 -1.27846362e+00  2.77885892e-01]
action:  [-5.77729974e

x_in: [ 1.          0.58778525 -1.27542081  0.294844  ]
action:  [0.00607201 0.00607201 0.00607201 0.00607201 0.00607201]
x_in: [ 1.          0.95105652 -1.27533431  0.29532229]
action:  [0.00971014 0.00971014 0.00971014 0.00971014 0.00971014]
x_in: [ 1.          0.95105652 -1.27529612  0.2955232 ]
action:  [0.00971253 0.00971253 0.00971253 0.00971253 0.00971253]
x_in: [ 1.          0.58778525 -1.27531645  0.295389  ]
action:  [0.0060785 0.0060785 0.0060785 0.0060785 0.0060785]
x_in: [ 1.00000000e+00  6.76165968e-14 -1.27538345e+00  2.94987081e-01]
action:  [0.00019604 0.00019604 0.00019604 0.00019604 0.00019604]
x_in: [ 1.         -0.58778525 -1.27546777  0.29448483]
action:  [-0.00568762 -0.00568762 -0.00568762 -0.00568762 -0.00568762]
x_in: [ 1.         -0.95105652 -1.27553406  0.29408526]
action:  [-0.00932478 -0.00932478 -0.00932478 -0.00932478 -0.00932478]
x_in: [ 1.         -0.95105652 -1.27555452  0.2939494 ]
action:  [-0.00932635 -0.00932635 -0.00932635 -0.00932635 -0.00932635

x_in: [ 1.          0.58778525 -1.27692737  0.28471536]
action:  [0.00595566 0.00595566 0.00595566 0.00595566 0.00595566]
x_in: [ 1.          0.95105652 -1.27690151  0.28485278]
action:  [0.00958978 0.00958978 0.00958978 0.00958978 0.00958978]
x_in: [ 1.          0.95105652 -1.27692097  0.28471717]
action:  [0.00958823 0.00958823 0.00958823 0.00958823 0.00958823]
x_in: [ 1.          0.58778525 -1.2769953   0.28425049]
action:  [0.00595033 0.00595033 0.00595033 0.00595033 0.00595033]
x_in: [ 1.00000000e+00 -8.82859861e-15 -1.27711280e+00  2.83521214e-01]
action:  [6.40841642e-05 6.40841642e-05 6.40841642e-05 6.40841642e-05
 6.40841642e-05]
x_in: [ 1.         -0.58778525 -1.27724532  0.28269796]
action:  [-0.00582326 -0.00582326 -0.00582326 -0.00582326 -0.00582326]
x_in: [ 1.         -0.95105652 -1.27735934  0.28198443]
action:  [-0.00946403 -0.00946403 -0.00946403 -0.00946403 -0.00946403]
x_in: [ 1.         -0.95105652 -1.27742874  0.2815416 ]
action:  [-0.00946915 -0.00946915 -0.009469

0

In [35]:
np.ones([2,2])

array([[1., 1.],
       [1., 1.]])